### Урок 4. Система управления базами данных MongoDB в Python

In [ ]:
from pymongo import MongoClient
import requests
from bs4 import BeautifulSoup as bs
from pprint import pprint
from pymongo import MongoClient
from pymongo.errors import InvalidDocument as idoc

In [ ]:
client = MongoClient('localhost', 27017)
db = client.headhunter
vacations = db.hh_vacations


In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
             
}
url = 'https://hh.ru/search/vacancy'


In [ ]:
def get_html(url, params=''):
    response = requests.get(url, headers=headers, params=params)
    return response

In [ ]:
def hh_content(response):
    dom = bs(response, 'lxml')
    items = dom.find_all('div', class_='vacancy-serp-item__layout')
    hh_vacancy = []

    for item in items:
        hh_vacancy.append(
            {
                'site': 'HeadHanter', # Название сайта
                'title': item.find('h3', class_='bloko-header-section-3').get_text(), # Название вакансии
                'link': item.find('a', class_='serp-item__title').get('href'), # Ссылка на вакансию
                'salary': item.find('span', class_='bloko-header-section-3'), # Зарплата
                'organization': item.find('div', class_='vacancy-serp-item-company').get_text() # Название компании
            }
        )
    for i in hh_vacancy:
      
        try:
            i['salary'] = i['salary'].text
        except:
            i['salary'] = None
        if i['salary']:
            salary_list = i['salary'].split(' ')
            if salary_list[0] == 'от':
                i['salary_min'] = salary_list[1]
                i['salary_max'] = None
            elif salary_list[0] == 'до':
                i['salary_min'] = None
                i['salary_max'] = salary_list[1]
            else:
                i['salary_min'] = salary_list[0]
                i['salary_max'] = salary_list[2]
            i['salary_currency'] = salary_list[-1]
        else:
            i['salary_min'] = None
            i['salary_max'] = None
            i['salary_currency'] = None
        i.pop('salary')
    return hh_vacancy

In [ ]:
def parser_hh():
    post = str(input('Введите название вакансии для парсинга: '))
    pages = int(input('Количество страниц для парсинга: '))
    html = get_html(url)
    if html.status_code == 200:
        vacancy = []
        for page in range(1, pages + 1):
            print(f'Парсим страницу {page}')
            html = get_html(url, params={'text': post, 'page': page})
            vacancy.extend(hh_content(html.text))
        hh_result = pd.DataFrame(vacancy)
    else:
        hh_result = html.status_code
    return hh_result

In [ ]:
hh_result = parser_hh()

### 1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.

In [ ]:
def insert_mongo(hh_result):
    for vac in hh_result:
        try:
            vacations.update_one({'_Id': vac['_Id']}, {'$set': vac}, upsert=True)
        except idoc:
            print(f'Error {idoc}')
            continue

In [ ]:
insert_mongo(hh_result)

### 2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты). Для тех, кто выполнил задание с Росконтролем - напишите запрос для поиска продуктов с рейтингом не ниже введенного или качеством не ниже введенного (то есть цифра вводится одна, а запрос проверяет оба поля).

In [ ]:
def search_by_salary():
    value = int(input('Введите значение заработной платы: '))
    res = vacations.find({'$or':
                              [{'валюта ЗП': 'руб.',
                                    '$or': [
                                        {'ЗП_min': {'$gt': value}},
                                        {'ЗП_max': {'$gt': value}},]
                                },
                                {'валюта ЗП': 'USD',
                                    '$or': [
                                        {'ЗП_min': {'$gt': value / 75}},
                                        {'ЗП_max': {'$gt': value / 75}},]
                                }]
                            })

    return list(res)

res = search_by_salary()
pprint(res)